In [ ]:
#IMPORT CONFIG FROM FILE .CONFIG
import configparser
import os
import re
from time import sleep
import database_functions as dbf
import telebot
from telebot import types

config_file_name = 'price_monitoring.config'
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
database = cfg['database_config']['database']
bot_token = cfg['telegram_config']['bot_token']

link_prefix = cfg['general_config']['link_prefix']
link_sufix = cfg['general_config']['link_sufix']
link_sufix_split = cfg['general_config']['link_sufix_split']

#CREATE TELEGRAM BOT
bot = telebot.TeleBot(bot_token)

In [ ]:
#tem que ajustar o processo da virgula no target_price - ok
#ajustar a consulta dos itens cadastrados
#criar como chave primaria o link, pois podem ter itens repetidos, na hora de cadastrar o produto - ok
#CRIAR ISSO COMO PRODUCT_ID E NAO COMO LINK - ok
#REPETIR O MESMO PARA A TABELA DE PRECOS - ok
#criar a lista de produtos cadastrados para servir de base para os comandos de consultar
#criar as atividades de alterar preco - ok
#criar a atividade de excluir produto
#criar atividade de enviar mensagem para o usuario quando atingir preco alvo
#transformar o price_monitoring em um pacote - ok
#criar atividade que varre todos os produtos cadastrados em todos os DBs de usuarios e compilar novos precos, atualizando o db de precos


In [ ]:
## VALIDATING / OTHER FUNCTIONS
#EXTRACT URL FROM TEXT
def get_url(text):
    try:
        url = re.search(r'(https?://[^\s]+)', text).group(1)
    except:
        url = None
    return url

#VALIDATE NUMBER
def validate_number(text):
    try:
        text = re.search(r'(^\d+(.\d+)*$)', text).group(1)
    except:
        text = None
    return str(text)

In [ ]:
## SQL FUNCTIONS
# FUNCTION TO RETRIEVE DATA FROM SQLITE DATABASE
def get_data_from_user_db(user_id):
    conn, cur = dbf.connect_to_database(database)
    query = "SELECT * FROM user_" + str(user_id)
    data = dbf.select_data(conn, cur, query)
    title = [description[0] for description in cur.description]
    conn.close()
    return data, title

# FUNCTION TO TRANSFORM SQLITE DB INTO DICTIONARY
def sqlite_table_to_dict(data, title):
    dict_data = []
    for row in data:
        dict_data.append(dict(zip(title, row)))
    return dict_data

# CONNECT TO DATABASE AND CREATE TABLE IF IT DOESN'T EXIST
def update_database(product_id, user, product_link, target_price):
    conn, cur = dbf.connect_to_database(database)
    columns = "product_id TEXT PRIMARY KEY, user TEXT, product_link TEXT, target_price REAL"
    table_name = "user_" + str(user)
    dbf.create_table(conn, cur, table_name, columns)
    sleep(0.5)
    data = "'" + str(product_id) + "','" + str(user) + "','" + product_link + "','" + str(target_price) + "'"
    dbf.insert_data(conn, cur, table_name, data)
    sleep(0.5)
    dbf.close_connection(conn)

# FUNCTION TO UPDATE PRICE IN DATABASE
def update_price_database(product_id, user, target_price):
    conn, cur = dbf.connect_to_database(database)
    table_name = "user_" + str(user)
    query = "UPDATE " + table_name + " SET target_price = '" + str(target_price) + "' WHERE product_id = '" + str(product_id) + "'"
    print(query)
    dbf.update_data(conn, cur, query)
    sleep(0.5)
    dbf.close_connection(conn)

In [ ]:
## BOT FUNCTIONS
user_dict = {}
class User:
    def __init__(self, chat_id):
        self.chat_id = chat_id
        self.url = None
        self.product_id = None
        self.product_link = None
        self.target_price = None

# HANDLE '/help' COMMAND
@bot.message_handler(commands=['help'])
def send_welcome(message):
    msg = bot.reply_to(message, """\
Ola, eu sou o bot de monitoramento de precos.
Envie o comando /start para iniciar.
""")

# HANDLE '/start' COMMAND
@bot.message_handler(commands=['start'])
def process_start_menu(message):
    msg_reply = "Ola, eu sou o bot de monitoramento de precos. O que deseja fazer?"
    try:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Cadastrar', 'Consultar', 'Sair')
        msg = bot.reply_to(message, msg_reply, reply_markup=markup)
        bot.register_next_step_handler(msg, process_start_menu_handler)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_start_menu_handler(message):
    try:
        answer = message.text
        if answer == u'Cadastrar':
            msg = bot.reply_to(message, 'Envie o link do produto que deseja monitorar')
            bot.register_next_step_handler(msg, process_url_step)
        elif answer == u'Consultar':
            #aqui tem que trocar mensagem e a funcao de consulta
            chat_id = message.chat.id
            data, title = get_data_from_user_db(chat_id)
            dict_data = sqlite_table_to_dict(data, title)
            msg_reply = 'Aqui esta a sua lista de itens: \n' + str(dict_data) + '\nInforme o que deseja realizar?'
            markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
            markup.add('Ajustar preco', 'Excluir item', 'Sair')
            msg = bot.reply_to(message, msg_reply, reply_markup=markup)
            bot.register_next_step_handler(msg, process_consult_step)
        elif answer == u'Sair':
            bot.reply_to(message, 'Obrigado, ate mais')
        else:
            raise Exception("Mensagem desconhecida")
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_url_step(message):
    try:
        chat_id = message.chat.id
        user = User(chat_id)
        user_dict[chat_id] = user
        url = get_url(message.text)
        if url is not None:
            user.url = url
            product_id = url.split(link_prefix)[1].split(link_sufix_split)[0]
            user.product_id = product_id
            product_link = link_prefix + product_id + link_sufix
            user.product_link = product_link
            msg = bot.reply_to(message, 'Informe o preco alvo do produto: \n (Usar o ponto como separador decimal: 1000.60)')
            bot.register_next_step_handler(msg, process_target_price_step)
        else:
            msg = bot.reply_to(message, 'O link nao e valido. Informe novamente o link do produto:')
            bot.register_next_step_handler(msg, process_url_step)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_step(message):
    try:
        chat_id = message.chat.id
        target_price = message.text
        if validate_number(str(target_price)) is None:
            msg = bot.reply_to(message, 'O preco deve ser um numero. Utilize ponto para separar casas decimais.\n Informar novamente o preco alvo do produto:')
            bot.register_next_step_handler(msg, process_target_price_step)
            return
        user = user_dict[chat_id]
        user.target_price = validate_number(str(target_price))
        update_database(user.product_id, user.chat_id, user.product_link, user.target_price)
        msg_reply = 'Product ID: ' + str(user.product_id) + '\nURL: ' + user.product_link + '\nPreco alvo: ' + str(user.target_price)
        bot.send_message(chat_id, msg_reply)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_consult_step(message):
    try:
        chat_id = message.chat.id
        answer = message.text
        if answer == u'Ajustar preco':
            msg = bot.reply_to(message, 'Envie o ID do produto que deseja ajustar o preco')
            bot.register_next_step_handler(msg, process_target_price_update_choose_item_step)
        elif answer == u'Excluir item':
            #aqui tem que trocar mensagem e a funcao de consulta
            msg = bot.reply_to(message, 'Envie o ID do produto que deseja excluir')
            #aqui vai a funcao de excluir######################################################
        elif answer == u'Sair':
            bot.reply_to(message, 'Obrigado, ate mais')
        else:
            raise Exception("Mensagem desconhecida")
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_update_choose_item_step(message):
    try:
        chat_id = message.chat.id
        user_dict['user'] = chat_id
        product_id = message.text
        user = user_dict['user']
        user_dict['product_id'] = product_id
        msg = bot.reply_to(message, 'Informe o preco alvo do produto: \n (Usar o ponto como separador decimal: 1000.60)')
        bot.register_next_step_handler(msg, process_target_price_update_step)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_update_step(message):
    try:
        chat_id = message.chat.id
        target_price = message.text
        product_id = user_dict['product_id']
        if validate_number(str(target_price)) is None:
            msg = bot.reply_to(message, 'O preco deve ser um numero. Utilize ponto para separar casas decimais.\nInformar novamente o preco alvo do produto:')
            bot.register_next_step_handler(msg, process_target_price_update_step)
            return
        user = user_dict['user']
        target_price = validate_number(str(target_price))
        product_id = user_dict['product_id']
        update_price_database(str(product_id), str(user), str(target_price))
        msg_reply = 'Product ID: ' + str(product_id) + '\nPreco alvo: ' + str(target_price) + '\nPreco atualizado com sucesso!'
        bot.send_message(chat_id, msg_reply)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')



In [ ]:
## BOT POLLING
# Enable saving next step handlers to file "./.handlers-saves/step.save".
# Delay=2 means that after any change in next step handlers (e.g. calling register_next_step_handler())
# saving will hapen after delay 2 seconds.
bot.enable_save_next_step_handlers(delay=2)

# Load next_step_handlers from save file (default "./.handlers-saves/step.save")
# WARNING It will work only if enable_save_next_step_handlers was called!
bot.load_next_step_handlers()

bot.infinity_polling()

In [ ]:

for key, value in user_dict.items():
    print(key, ' : ', value)

In [ ]:
user_id = 164914351
data, title = get_data_from_user_db(user_id)
dict_data = sqlite_table_to_dict(data, title)
dict_data

In [ ]:
links = [
    r'https://www.coupang.com/vp/products/6239304413?itemId=12584859912&vendorItemId=79852847425&sourceType=SDP_MID_BANNER&clickEventId=9f78756b-711f-48f4-aeae-f8ccf5ee3c02&isAddedCart=',
    r'https://www.coupang.com/vp/products/6366671555?isAddedCart=',
    r'https://www.coupang.com/vp/products/5056493492?isAddedCart=',
    r'https://www.coupang.com/vp/products/5835511110?isAddedCart=',
    r'https://www.coupang.com/vp/products/6060667011?isAddedCart=',
    r'https://www.coupang.com/vp/products/2530237?isAddedCart=',
    r'https://www.coupang.com/vp/products/266030794?isAddedCart=',
    r'https://www.coupang.com/vp/products/4978269600?isAddedCart='

]